#                                    Data Analytics Challenge
### por *Cristopher García*

A continuación presentaré el proceso que seguí para completar el reto presentado de realizar un **Análisis de Sentimieto** para visualizar el movimiento de la **Polaridad** en los últimos 9 días en torno a la expresión **"Panamá"** con tweets obtenidos accediento al *API* de **Twitter**.  ***¡Reto aceptado!***

### Las Herramientas

La necesidad de extraer datos no-estructurados de manera sistemática desde la web conlleva a usar un lenguaje de programación, por lo que he elegido **R**.  Todo el análisis fue creado en **R Studio** y me apoyé principalmente en las siguientes librerías:  **twitteR** para el web scraping y **ggplot2** (parte de tidyverse) para la visualización.  Las demás librerías ayudan a la manipulación de los datos.  

En la siguiente celda puede instalar las librerías requeridas.


In [ ]:
##Instalación de Librerías
install.packages("twitteR")
install.packages("purrr")
install.packages("dplyr")
install.packages("ROAuth")
install.packages("RCurl")
install.packages("stringr")
install.packages("tidyverse")
install.packages("lubridate")

Ejecutar la siguiente celda para cargar las librerías requeridas.

In [ ]:
##Carga de Librerías
library(twitteR)
library(purrr)
library(dplyr)
require("ROAuth")
require("RCurl")
library(plyr)
library(stringr)
library(tidyverse)
library(lubridate)

### El lexicon 

Para que la función que utilizaremos para evaluar la polaridad de los *tweets* funcione, esta necesita comparar cada palabra con un *lexicon* (listas de palabras positivas y negativas).  Decidí que lo más apropiado sería utilizar uno en español considerando que lo más probable es que la mayoría de los *tweets* en la nación y en la región estarán en este idioma, y por lo que sé, no hay razon para que la comunidad internacional de habla inglesa este conversando sobre nuestro país en estos momentos.

En la *web* no pude encontrar un *lexicon* en español pero sí conseguí uno en inglés, publicado por ***Bing Liu, Minqing Hu y Junsheng Cheng***, el cual procedí a traducir.  

Descargar los archivos del *lexicon* desde el repositorio y colocarlo en su carpeta de **Jypiter** antes de ejecutar la siguiente celda, la cual cargará el lexicon. 

In [ ]:
## carga del lexicon (cargará siempre que los archivos se encuentren en su directorio de trabajo activo ó working directory)
pos.words <- scan(".//palabras_positivas.txt", what = "character", comment.char = ";")
neg.words <- scan(".//palabras_negativas.txt", what = "character", comment.char = ";")

### La función evaluadora

La siguiente función, publicada por la plataforma de elearning **edureka!** en su canal de ***Youtube***, es el músculo del código.  Hay muchas otras funciones publicadas de variada complejidad y esta me pareció adecuada por su sencillez y eficacia. La misma evalua la polaridad de un texto despojandolo de carácteres especiales y dígitos, cambiando todas las letras a minúsculas, separándo el texto en palabras individuales, comparándo cada palabra con el lexicon para determinar si la misma es positiva, negativa o neutra, para finalmente hacer una sumatoria y devolver la evalación para todo el texto.  

Ejecutar la siguiente celda para cargar la función. 

In [ ]:
## función de evaluacion de sentimiento
score_sentiment<- function(sentences, pos.words, neg.words, .progress="none") 
{
        require(plyr)
        require(stringr)
        scores <- laply(sentences, function(sentence, pos.words, neg.words){
                sentence <- gsub("[[:punct:]]", "", sentence)
                sentence <- gsub("[[:cntrl:]]", "", sentence)
                sentence <- gsub("\\d+", "", sentence)
                sentence <- tolower(sentence)
                word.list <- str_split(sentence, "\\s+")
                words <- unlist(word.list)
                pos.matches <- match(words, pos.words)
                neg.matches <- match(words, neg.words)
                pos.matches <- !is.na(pos.matches)
                neg.matches <- !is.na(neg.matches)
                score <- sum(pos.matches) - sum(neg.matches)
                return(score)
        }, pos.words,neg.words, .progress = .progress)
        scores.df <- data.frame(score=scores, text=sentences)
        return(scores.df)
}


### Conectando con Twitter

El siguiente código nos permite conectarnos al *API* de **Twitter** para acceder a la información pública.  Provea los datos de su aplicación en **Twitter** para proseguir.

In [ ]:
## conexión al API de Twitter
api_key<-""
api_secret_key<-""
access_token<-""
access_token_secret<-""
setup_twitter_oauth(api_key,api_secret_key,access_token,access_token_secret)

### Aquí se rompe la reproducibilidad

Aunque los lenguajes de programación nos brindan una mayor capacidad de reproducibilidad que otras herramientas utilizadas para el análisis de datos, el poco control que tenemos sobre la muestra y la limitante que impone **Twitter** para acceder a la data de un máximo de siete días en el pasado, hace que pudieramos obtener resultados ligeramente diferentes cada vez que ejecutemos el código, incluso en un mismo día. 

Aún así, de querer tener un registro histórico a partir de la fecha se puede correr el código cada día y archivar el resultado.  

A continuación definimos las variables de fechas que utilizaremos a partir de la fecha del sistema.

In [ ]:
#variables de ultimas 7 fechas
hoy <- as.character(Sys.Date())
hoym1 <- as.character(Sys.Date()-1)
hoym2 <- as.character(Sys.Date()-2)
hoym3 <- as.character(Sys.Date()-3)
hoym4 <- as.character(Sys.Date()-4)
hoym5 <- as.character(Sys.Date()-5)
hoym6 <- as.character(Sys.Date()-6)

### Web Scraping

Apoyándonos en la librería **twitteR** procedemos a obtener los tweets en torno a *#Panama*.

In [ ]:

#obtencion de los tweets de cada día
tweets_hoy <- searchTwitter("#Panama", n=143, since = hoy)
tweets_hoym1 <- searchTwitter("#Panama", n=143, since = hoym1 , until = hoy)
tweets_hoym2 <- searchTwitter("#Panama", n=143, since = hoym2 , until = hoym1)
tweets_hoym3 <- searchTwitter("#Panama", n=143, since = hoym3 , until = hoym2)
tweets_hoym4 <- searchTwitter("#Panama", n=143, since = hoym4 , until = hoym3)
tweets_hoym5 <- searchTwitter("#Panama", n=143, since = hoym5 , until = hoym4)
tweets_hoym6 <- searchTwitter("#Panama", n=143, since = hoym6 , until = hoym5)

A continuación procedemos a preparar la data para que sea evaluada por la función principal, hacemos la evaluación y preparamos el resultado para la visualización.

In [ ]:
#cambio de structura de lista a data frame
df_hoy <- tbl_df(map_df(tweets_hoy, as.data.frame))
df_hoym1 <- tbl_df(map_df(tweets_hoym1, as.data.frame))
df_hoym2 <- tbl_df(map_df(tweets_hoym2, as.data.frame))
df_hoym3 <- tbl_df(map_df(tweets_hoym3, as.data.frame))
df_hoym4 <- tbl_df(map_df(tweets_hoym4, as.data.frame))
df_hoym5 <- tbl_df(map_df(tweets_hoym5, as.data.frame))
df_hoym6 <- tbl_df(map_df(tweets_hoym6, as.data.frame))


#evaluación de sentimiento de los tweets
score_hoy <- score_sentiment(df_hoy$text, pos.words, neg.words)
score_hoym1<- score_sentiment(df_hoym1$text, pos.words, neg.words)
score_hoym2 <- score_sentiment(df_hoym2$text, pos.words, neg.words)
score_hoym3 <- score_sentiment(df_hoym3$text, pos.words, neg.words)
score_hoym4 <- score_sentiment(df_hoym4$text, pos.words, neg.words)
score_hoym5 <- score_sentiment(df_hoym5$text, pos.words, neg.words)
score_hoym6 <- score_sentiment(df_hoym6$text, pos.words, neg.words)

#integración en data frame único
tweets_df <- rbind(df_hoym6,df_hoym5,df_hoym4, df_hoym3, df_hoym2,df_hoym1, df_hoy)
scores_df <- rbind(score_hoym6, score_hoym5, score_hoym4, score_hoym3, score_hoym2, score_hoym1, score_hoy)
single_df <- cbind(tweets_df[,5], scores_df$score)
single_df$Fechasolo <- date(single_df[,1])
meanhoy<- mean(score_hoy$score)       
meanhoy<- rep(meanhoy, 143)
meanhoym1<-mean(score_hoym1$score)
meanhoym1<-rep(meanhoym1,143)
meanhoym2<-mean(score_hoym2$score)
meanhoym2<-rep(meanhoym2, 143)
meanhoym3<-mean(score_hoym3$score)
meanhoym3<-rep(meanhoym3, 143)
meanhoym4<-mean(score_hoym4$score)
meanhoym4<-rep(meanhoym4, 143)
meanhoym5<-mean(score_hoym5$score)
meanhoym5<-rep(meanhoym5, 143)
meanhoym6<-mean(score_hoym6$score)
meanhoym6<-rep(meanhoym6, 143)
meancol<-c(meanhoym6,meanhoym5,meanhoym4, meanhoym3, meanhoym2, meanhoym1, meanhoy)
single_df$mean<- meancol
names(single_df) <- c("Fecha", "Score", "Fechasolo", "Promedio")



### Visualizaciones

La presentación de la información la podemos y debemos adecuar a nuestra audiencia.

Con la primera visualización pretendo mostrar la información más importante de una manera muy clara y consisa al presentar sólo la polaridad promedio de cada día.  Para ayudar a la comparación de un día a otro se ha agregado una escala de color que inmediatamente muestra el sentimiento general para cada día.

In [ ]:
ggplot(single_df, aes(x=as.factor(Fechasolo), y=Score, color=Promedio)) + scale_color_gradient(low = "red", high = "orange") + labs(x="", y= "Polaridad", title = "Análisis de Sentimiento", subtitle = "'#Panama'") + geom_boxplot(fill="white",outlier.size = 2,outlier.shape = 21,outlier.fill = "White", size=0.9, color="white") + theme_classic() + theme(panel.border = element_rect(fill = NA, color = "black", size = 0.8), plot.title = element_text(hjust = 0.5, size = 20) , plot.subtitle = element_text(hjust = 0.5, size = 13), axis.title = element_text(size = 15)) + geom_point(stat = "summary", fun="mean", shape= 16, size=20) 


La gráfica de cajas y bigotes presenta mucha más información sobre nuestros datos que podría ser de interés para un público con más conocimientos estadísticos. 

El símbolo X en la caja nos muestra la polaridad promedio para cada día y sólo con eso cualquier persona puede tener una buena idea de como se ha movido la polaridad a través de los últimos siete días.  Así mismo tenemos una mejor visión de la variabilidad en la data y de las observaciones atípicas (*outliers*).


In [ ]:
vis_base <- ggplot(single_df, aes(x=as.factor(Fechasolo), y=Score)) + geom_boxplot()
vis_base + labs(x="", y= "Polaridad", title = "Análisis de Sentimiento", subtitle = "'#Panama'") + geom_boxplot(fill=1300,outlier.size = 2,outlier.shape = 21,outlier.fill = "White", size=0.9) + theme_bw() + theme(panel.border = element_rect(fill = NA, color = "black", size = 0.8), plot.title = element_text(hjust = 0.5, size = 20) , plot.subtitle = element_text(hjust = 0.5, size = 13), axis.title = element_text(size = 15)) + geom_point(stat = "summary", fun= "mean", shape= 4, size=2)



La data muestra que el sentimiento en torno a ***#Panamá*** varía entre neutral a negativo, algo que hace mucho sentido dada la realidad de los efectos del COVID-19, la tragedia en tierras altas, y hasta el mal resultado deportivo ante **Estados Unidos**.

Espero que el día de la entrevista técnica podamos ver resultamos un poco más positivos...

### Referencias

edureka!. Sentiment Analysis in R | Sentiment Analysis of Twitter Data | Data Science Training. https://www.youtube.com/watch?v=-JW6_kcHDj4&t=2285s

Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews."

Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing and Comparing Opinions on the Web."